# Tarea Semana 11

# Elberth Adrián Garro Sánchez (1-1644-0594)

# Imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import SGD
from tensorflow.keras.models import model_from_json
from plotly.subplots import make_subplots

# Load Data

In [ ]:
print(os.listdir('../input'))

In [ ]:
# Since data is duplicated (see: https://www.kaggle.com/alxmamaev/flowers-recognition/discussion/188011)
# Using this path:
final_path = "/kaggle/input/flowers-recognition/flowers/flowers"
classes = os.listdir(final_path)
classes

In [ ]:
image_size = (128, 128)
batch_size = 256

# https://keras.io/api/preprocessing/image/#image_dataset_from_directory-function

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    final_path,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    class_names=classes
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    final_path,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    class_names=classes
)

# Data review

In [ ]:
len(train_ds)

In [ ]:
train_ds

In [ ]:
len(val_ds)

In [ ]:
val_ds

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(classes[np.argmax(labels[i])])
        plt.axis("off")

# Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.5),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

# Model

## Perceptron Architecture

In [ ]:
def make_model(input_shape, num_classes):
    
    inputs = keras.Input(shape=input_shape)

    # Image augmentation block
    x = data_augmentation(inputs)

    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    outputs = layers.Dense(units, activation=activation)(x)
    
    return keras.Model(inputs, outputs)

In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=len(classes))
model.summary()

## Compilation

In [ ]:
model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

## Training

In [ ]:
epochs = 40

# https://keras.io/api/callbacks/

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

history = model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

## Model Training History

In [ ]:
print(history.history.keys())

In [ ]:
history_fig = make_subplots(rows=1, cols=2)

history_fig.add_trace(go.Scatter(
    y=history.history['loss'],
    mode='lines+markers',
    name='training loss'
), row=1, col=1)

history_fig.add_trace(go.Scatter(
    y=history.history['val_loss'],
    mode='lines+markers',
    name='validation loss'
), row=1, col=1)


history_fig.add_trace(go.Scatter(
    y=history.history['accuracy'],
    mode='lines+markers',
    name='training accuracy'
), row=1, col=2)

history_fig.add_trace(go.Scatter(
    y=history.history['val_accuracy'],
    mode='lines+markers',
    name='validation accuracy'
), row=1, col=2)

history_fig.update_xaxes(title_text='Epoch')

history_fig.update_layout(
    title_text="Training History Metrics",
    legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1)
)

history_fig.show()

## Saving

In [ ]:
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("Saved model to disk")

## Loading

In [ ]:
# Load json and create model:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model:
loaded_model.load_weights("model.h5")

print("Loaded model from disk")

## Testing Predictions

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in val_ds.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        pred = loaded_model.predict(np.array([images[i].numpy().astype("uint8")]))
        title = 'Prediction: ' + classes[np.argmax(pred)]
        title += '\nReality: ' + classes[np.argmax(labels[i])]
        plt.title(title)
        plt.axis("off")